In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.inspection import DecisionBoundaryDisplay

In [ ]:
# 1. Dados: Círculos
# noise=0.1: Ruído controlado para manter a forma nítida
X, y = make_circles(n_samples=600, noise=0.1, factor=0.4, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
print(y)

In [ ]:
# --- RAMO 1: Random Forest (Bagging) ---
# Limitada (max_depth=2). Tenta aproximar círculo com retângulos.
# Resultado: Fronteira quadrada/diamante.
model_paralelo = RandomForestClassifier(n_estimators=50, max_depth=2, random_state=42)
model_paralelo.fit(X_train, y_train)
acc_paralelo = model_paralelo.score(X_test, y_test)

In [ ]:
# --- RAMO 2: Gradient Boosting (Boosting) ---
# Sequencial. Corrige os erros de forma (viés).
# Resultado: Círculo quase perfeito.
model_sequencial = GradientBoostingClassifier(n_estimators=50, max_depth=2, random_state=42)
model_sequencial.fit(X_train, y_train)
acc_sequencial = model_sequencial.score(X_test, y_test)

In [ ]:
# --- RAMO 3: Voting Classifier (Heterogêneo Ruim) ---
# AQUI ESTAVA O PROBLEMA: O KNN era bom demais.
# Vamos usar 2 modelos LINEARES + 1 Árvore fraca.
# A "maioria" do comité agora acha que o mundo é plano (linear).
clf1 = LogisticRegression(random_state=42)
clf2 = SVC(kernel='linear', probability=True, random_state=42) # Outro linear!
clf3 = DecisionTreeClassifier(max_depth=2, random_state=42)

model_heterogeneo = VotingClassifier(
    estimators=[('lr', clf1), ('svc_lin', clf2), ('tree', clf3)],
    voting='soft'
)
model_heterogeneo.fit(X_train, y_train)
acc_heterogeneo = model_heterogeneo.score(X_test, y_test)

In [ ]:
# 4. Visualização
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
models = [model_paralelo, model_sequencial, model_heterogeneo]
names = [
    f"Random Forest\n(Forma de Diamante)\nAcc: {acc_paralelo:.2f}",
    f"Gradient Boosting\n(Forma Circular)\nAcc: {acc_sequencial:.2f}",
    f"Voting (2 Lineares + 1 Tree)\n(Colapso do Modelo)\nAcc: {acc_heterogeneo:.2f}"
]

for i, model in enumerate(models):
    # Fronteira
    DecisionBoundaryDisplay.from_estimator(
        model, X, cmap=plt.cm.RdBu, alpha=0.6, ax=axes[i], response_method="predict"
    )
    # Pontos
    axes[i].scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdBu, edgecolors='k', s=20)
    axes[i].set_title(names[i], fontsize=12, fontweight='bold')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print(f"--- RESULTADO FINAL ---")
print(f"GBM ({acc_sequencial:.2f}) vence de lavada o Voting ({acc_heterogeneo:.2f}).")
print("Lição: Juntar vários 'especialistas' errados (lineares) cria uma democracia burra.")